In [7]:
from dotenv import load_dotenv
import os


load_dotenv(dotenv_path=".env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [8]:
# ロール定義
ROLES = {
    "1": {
        "name": "一般知識エキスパート",
        "description": "幅広い分野の一般的な質問に答える",
        "details": "幅広い分野の一般的な質問に対して、正確で分かりやすい回答を提供してください"
    },
    "2": {
        "name": "生成AI製品エキスパート",
        "description": "生成AIや関連製品、技術に関する専門的な質問に答える",
        "details": "生成AIや関連製品、技術に関する専門的な質問に対して、最新の情報と深い洞察を持って回答してください"
    },
    "3": {
        "name": "カウンセラー",
        "description": "個人的な悩みや問題に対してサポートを提供する",
        "details": "個人的な悩みや問題に対して、共感的で支援的な回答を提供し、可能であれば適切なアドバイスも行なってください"
    }
}

In [9]:
# ステートの定義
import operator
from typing import Annotated

from pydantic import BaseModel, Field


class State(BaseModel):
    query: str = Field(..., description="ユーザーからの質問")
    current_role: str = Field(default="", description="選定された回答ロール")
    messages: Annotated[list[str], operator.add] = Field(
        default_factory=list, description="回答履歴"
    )
    current_judge: bool = Field(
        default=False, description="品質チェックの結果"
    )
    judgement_reason: str = Field(
        default="", description="品質チェックの判定理由"
    )

In [10]:
# chat modelの初期化
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
llm = llm.configurable_fields(max_tokens=ConfigurableField(id="max_tokens"))

In [11]:
# ノードの定義
from typing import Any

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def selection_node(state: State) -> dict[str, Any]:
    query = state.query
    role_options = "\n".join([f"{k}. {v['name']}: {v['description']}" for k, v in ROLES.items()])
    prompt = ChatPromptTemplate.from_template(
""" 質問を分析し、最も適切な回答担当ロールを選択してください。

選択肢：
{role_options}

回答は選択肢の番号（1、2、または3）のみを返してください。

質問: {query}
""".strip()
    )
    chain = prompt | llm.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
    role_number = chain.invoke({"role_options": role_options, "query": query})

    selected_role = ROLES[role_number.strip()]["name"]
    return {"current_role": selected_role}

In [12]:
# answeringノードの実装
def answering_note(state: State) -> dict[str, Any]:
    query = state.query
    role = state.current_role
    role_details = "\n".join([f" - {v['name']}: {v['details']}" for v in ROLES.values()])
    prompt = ChatPromptTemplate.from_template(
""" あなたは{role}として回答してください。以下の質問に対して、あなたの役割に基づいた適切な回答を提供してください。

役割の詳細：
{role_details}

質問: {query}

回答:""".strip()
    )
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"role": role, "role_details": role_details, "query": query})
    return {"messages": [answer]}

In [13]:
# checkノードの実装
class Judgement(BaseModel):
    reason: str = Field(default="", description="判定理由")
    judge: bool = Field(default=False, description="判定結果")

def check_node(state: State) -> dict[str, Any]:
    query = state.query
    answer = state.messages[-1]
    prompt = ChatPromptTemplate.from_template(
""" 以下の回答の品質をチェックし、問題がある場合は'False'、問題がない場合は'True'を回答してください。また、その判断理由も説明してください。

ユーザーからの質問: {query}
回答: {answer}
""".strip()
    )
    chain = prompt | llm.with_structured_output(Judgement)
    result: Judgement = chain.invoke({"query": query, "answer": answer})

    return {"current_judge": result.judge, "judgement_reason": result.reason}

In [14]:
# グラフの作成
from langgraph.graph import StateGraph

workflow = StateGraph(State)

In [15]:
# ノードの追加
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_note)
workflow.add_node("check", check_node)

In [16]:
# エッジの定義
workflow.set_entry_point("selection")
workflow.add_edge("selection", "answering")
workflow.add_edge("answering", "check")

In [17]:
# 条件付きエッジの実装
from langgraph.graph import END

workflow.add_conditional_edges(
    "check",
    lambda state: state.current_judge,
    {True: END, False: "answering"}
)

In [18]:
# グラフのコンパイル
compiled = workflow.compile()

In [19]:
# グラフの実行
initial_state = State(query="生成AIについて教えてください。")
result = compiled.invoke(initial_state)
result

{'query': '生成AIについて教えてください。',
 'current_role': '生成AI製品エキスパート',
 'messages': ['生成AIとは、人工知能の一分野であり、特にデータを基に新しいコンテンツを生成する能力を持つ技術を指します。これには、テキスト、画像、音声、動画など、さまざまな形式のコンテンツが含まれます。生成AIは、機械学習アルゴリズム、特に深層学習を利用して、既存のデータからパターンを学習し、それに基づいて新しいデータを生成します。\n\n### 主な技術と応用\n1. **自然言語処理（NLP）**: テキスト生成や翻訳、要約などに使用されます。例えば、GPT（Generative Pre-trained Transformer）シリーズは、文章を生成する能力に優れています。\n\n2. **画像生成**: GAN（Generative Adversarial Networks）やVQ-VAE（Vector Quantized Variational Autoencoders）などの技術を用いて、リアルな画像を生成することができます。DALL-EやMidjourneyなどがその例です。\n\n3. **音声生成**: 音声合成技術を用いて、人間の声に似た音声を生成することができます。これにより、音声アシスタントやナレーションなどが実現されています。\n\n4. **動画生成**: まだ発展途上ですが、AIを用いて短い動画を生成する技術も進化しています。\n\n### 利点と課題\n- **利点**: 生成AIは、クリエイティブな作業を支援し、効率を向上させる可能性があります。また、個別化されたコンテンツの生成や、データの分析から新しい洞察を得る手助けをします。\n\n- **課題**: 一方で、生成AIには倫理的な問題や著作権の問題、フェイクニュースの生成といったリスクも伴います。生成されたコンテンツの信頼性や透明性を確保することが重要です。\n\n### 未来の展望\n生成AIは、今後もさまざまな分野での応用が期待されており、特に教育、エンターテインメント、マーケティングなどでの利用が進むでしょう。また、技術の進化に伴い、より高品質で多様なコンテンツが生成されることが予想されます。\n\nこのように、生成AIは非常に多岐に

In [20]:
# 結果の表示
print(result["messages"][-1])

生成AIとは、人工知能の一分野であり、特にデータを基に新しいコンテンツを生成する能力を持つ技術を指します。これには、テキスト、画像、音声、動画など、さまざまな形式のコンテンツが含まれます。生成AIは、機械学習アルゴリズム、特に深層学習を利用して、既存のデータからパターンを学習し、それに基づいて新しいデータを生成します。

### 主な技術と応用
1. **自然言語処理（NLP）**: テキスト生成や翻訳、要約などに使用されます。例えば、GPT（Generative Pre-trained Transformer）シリーズは、文章を生成する能力に優れています。

2. **画像生成**: GAN（Generative Adversarial Networks）やVQ-VAE（Vector Quantized Variational Autoencoders）などの技術を用いて、リアルな画像を生成することができます。DALL-EやMidjourneyなどがその例です。

3. **音声生成**: 音声合成技術を用いて、人間の声に似た音声を生成することができます。これにより、音声アシスタントやナレーションなどが実現されています。

4. **動画生成**: まだ発展途上ですが、AIを用いて短い動画を生成する技術も進化しています。

### 利点と課題
- **利点**: 生成AIは、クリエイティブな作業を支援し、効率を向上させる可能性があります。また、個別化されたコンテンツの生成や、データの分析から新しい洞察を得る手助けをします。

- **課題**: 一方で、生成AIには倫理的な問題や著作権の問題、フェイクニュースの生成といったリスクも伴います。生成されたコンテンツの信頼性や透明性を確保することが重要です。

### 未来の展望
生成AIは、今後もさまざまな分野での応用が期待されており、特に教育、エンターテインメント、マーケティングなどでの利用が進むでしょう。また、技術の進化に伴い、より高品質で多様なコンテンツが生成されることが予想されます。

このように、生成AIは非常に多岐にわたる可能性を秘めた技術であり、今後の発展が楽しみです。


In [21]:
import operator
from typing import Annotated, Any
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class State(BaseModel):
    query: str
    messages: Annotated[list[BaseMessage], operator.add] = Field(default=[])

# メッセージを追加するノード関数
def add_message(state: State) -> dict[str, Any]:
    additional_messages = []
    if not state.messages:
        additional_messages.append(
            SystemMessage(content="あなたは最小限の応答をする対話エージェントです。")
        )
    additional_messages.append(HumanMessage(content=state.query))
    return {"messages": additional_messages}

# LLMからの応答を追加するノード関数
def llm_response(state: State) -> dict[str, Any]:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
    ai_message = llm.invoke(state.messages)
    return {"messages": [ai_message]}

In [22]:
from pprint import pprint
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.base import BaseCheckpointSaver

def print_checkpoint_dump(checkpointer: BaseCheckpointSaver, config: RunnableConfig):
    checkpoint_tuple = checkpointer.get_tuple(config)

    print("チェックポイントデータ:")
    pprint(checkpoint_tuple.checkpoint)
    print("\nメタデータ:")
    pprint(checkpoint_tuple.metadata)

In [23]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# グラフを設定
graph = StateGraph(State)
graph.add_node("add_message", add_message)
graph.add_node("llm_response", llm_response)

graph.set_entry_point("add_message")
graph.add_edge("add_message", "llm_response")
graph.add_edge("llm_response", END)

# チェックポインターを設定
checkpointer = MemorySaver()

# グラフをコンパイル
compiled_graph = graph.compile(checkpointer=checkpointer)

In [24]:
config = {"configurable": {"thread_id": "example-1"}}
user_query = State(query="私の好きなものはずんだ餅です。教えてください。")
first_response = compiled_graph.invoke(user_query, config=config)
first_response

{'query': '私の好きなものはずんだ餅です。教えてください。',
 'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='私の好きなものはずんだ餅です。教えてください。', additional_kwargs={}, response_metadata={}),
  AIMessage(content='ずんだ餅は、枝豆をすりつぶして作る甘いペーストを乗せた餅です。主に東北地方の名物です。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 46, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CgOws2zrkuFwdhBE4mmZsr42hzBEE', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--9061b075-9145-4993-afcd-1ce1e86e7cf9-0', usage_metadata={'input_tokens': 46, 'output_tokens': 40, 'total_token

In [25]:
for checkpoint in checkpointer.list(config):
    print(checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0cb52d-a1ec-666c-8002-0c1bf6b45d7a'}}, checkpoint={'v': 4, 'ts': '2025-11-27T05:35:15.569193+00:00', 'id': '1f0cb52d-a1ec-666c-8002-0c1bf6b45d7a', 'channel_versions': {'__start__': '00000000000000000000000000000002.0.09952418539396579', 'query': '00000000000000000000000000000002.0.09952418539396579', 'messages': '00000000000000000000000000000004.0.5712828253570692', 'branch:to:add_message': '00000000000000000000000000000003.0.17927004844485583', 'branch:to:llm_response': '00000000000000000000000000000004.0.5712828253570692'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000001.0.5393510942677086'}, 'add_message': {'branch:to:add_message': '00000000000000000000000000000002.0.09952418539396579'}, 'llm_response': {'branch:to:llm_response': '00000000000000000000000000000003.0.17927004844485583'}}, 'updated_channels': ['messages'], 'channel_

In [26]:
print_checkpoint_dump(checkpointer, config)

チェックポイントデータ:
{'channel_values': {'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}),
                                 HumanMessage(content='私の好きなものはずんだ餅です。教えてください。', additional_kwargs={}, response_metadata={}),
                                 AIMessage(content='ずんだ餅は、枝豆をすりつぶして作る甘いペーストを乗せた餅です。主に東北地方の名物です。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 46, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CgOws2zrkuFwdhBE4mmZsr42hzBEE', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--9061b075-9145-4993-afcd-1ce1e86e7cf9-0', usage_metada

In [27]:
user_query = State(query="私の好物は何か覚えてる？")
second_response = compiled_graph.invoke(user_query, config)
second_response

{'query': '私の好物は何か覚えてる？',
 'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='私の好きなものはずんだ餅です。教えてください。', additional_kwargs={}, response_metadata={}),
  AIMessage(content='ずんだ餅は、枝豆をすりつぶして作る甘いペーストを乗せた餅です。主に東北地方の名物です。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 46, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CgOws2zrkuFwdhBE4mmZsr42hzBEE', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--9061b075-9145-4993-afcd-1ce1e86e7cf9-0', usage_metadata={'input_tokens': 46, 'output_tokens': 40, 'total_tokens': 86, 'in

In [28]:
for checkpoint in checkpointer.list(config):
    print(checkpoint)

CheckpointTuple(config={'configurable': {'thread_id': 'example-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0cb534-01c1-6d66-8006-e24ebfe5d0b4'}}, checkpoint={'v': 4, 'ts': '2025-11-27T05:38:06.679352+00:00', 'id': '1f0cb534-01c1-6d66-8006-e24ebfe5d0b4', 'channel_versions': {'__start__': '00000000000000000000000000000006.0.41802013364624735', 'query': '00000000000000000000000000000006.0.41802013364624735', 'messages': '00000000000000000000000000000008.0.7042257069208121', 'branch:to:add_message': '00000000000000000000000000000007.0.07345397204903292', 'branch:to:llm_response': '00000000000000000000000000000008.0.7042257069208121'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000005.0.3291508034329029'}, 'add_message': {'branch:to:add_message': '00000000000000000000000000000006.0.41802013364624735'}, 'llm_response': {'branch:to:llm_response': '00000000000000000000000000000007.0.07345397204903292'}}, 'updated_channels': ['messages'], 'channel_

In [29]:
config = {"configurable": {"thread_id": "example-2"}}
user_query = State(query="私の好物は何？")
other_thread_response = compiled_graph.invoke(user_query, config)
other_thread_response

{'query': '私の好物は何？',
 'messages': [SystemMessage(content='あなたは最小限の応答をする対話エージェントです。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='私の好物は何？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='わかりません。あなたの好物は何ですか？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 36, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b547601dbd', 'id': 'chatcmpl-CgP1qUWzoGJWxWvf5M9REVJ5ofItM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--f3407ca9-2de2-47eb-8553-1ecce9b889f5-0', usage_metadata={'input_tokens': 36, 'output_tokens': 14, 'total_tokens': 50, 'input_token_details': {'audio': 0, 'cache_read'